<a href="https://colab.research.google.com/github/AlexKressner/Vorlesung-Industrielles-Management/blob/main/Produktionsplanung_DatenEinlesen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Produktionsplanung
## Capacitated Lot Sizing Problem (CLSP)

## Modell

### Indexmengen
$p \in P$ : Menge der Produkte

$t \in T$ : Menge der Planungsperioden (Wochen)



### Parameter
$cap_t$ : Kapazität der Abfüllanlage in Woche $t$

$lc_{p}$ : Lagerkosten für Produkt $p$

$rc_{p}$ : Rüstkosten für Produkt $p$

$bz_p$ : Bearbeitungszeit für Produkt $p$

$rz_p$ : Rüstzeit für Produkt $p$

$d_{pt}$ : Nachfrage nach Produkt $p$ in Woche $t$

### Variablen
$Q_{pt} \ge 0$ : Produktionsmenge von Produkt $p$ in Woche $t$

$B_{pt} \ge 0$ : Lagerbestand von Produkt $p$ in Woche $t$

$R_{pt} \in \{0,1\}$ : Binäre Rüstvariable

### Zielfunktion
Min $K = \sum_{p,t} (rc_p * R_{p,t} + lc_p * B_{pt})$

### Nebenbedingungen

**(1) Lagerbilanzgleichung**

$B_{pt} = B_{p,t-1} + Q_{pt} - d_{pt}$

$∀ p, t$

**(2) Produktionskapazitäten**

$\sum_{p} (bz_p * Q_{pt} + rz_p * R_{pt}) \le pcap_{t}$

$∀ t$

**(3) Koppelung Produktions- mit Rüstentscheidung**

$Q_{pt} \le \frac{cap_t}{bz_p} * R_{pt} $

$∀ p, t$

## Implementierung

In [ ]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

In [ ]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Datenaufbereitung


1.   Problemrelevante Daten in Google-Drive laden
2.   Google-Drive mit Colab-Notebook verbinden
3.   Daten mit `pandas` laden



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Ordner finden
! ls drive/MyDrive/Industrielles_Management/Daten/Produktionsplanung

In [ ]:
# Pfad zurückgeben
! cd drive/MyDrive/Industrielles_Management/Daten/Produktionsplanung && pwd

In [ ]:
# Daten laden
import pandas as pd

In [ ]:
path = "/content/drive/MyDrive/Industrielles_Management/Daten/Produktionsplanung"

In [ ]:
# Nachfrage
nachfrage = pd.read_csv(f"{path}/Nachfrage.csv", sep=";")

In [ ]:
nachfrage

In [ ]:
# Kosten & Zeiten
kosten_zeiten = pd.read_csv(f"{path}/Kosten_Zeiten.csv", sep=";", decimal=",")

In [ ]:
kosten_zeiten

## Indexmengen

In [ ]:
P = nachfrage["Produkt"].unique().tolist() # Menge der Produkte

In [ ]:
T = nachfrage["Woche"].unique().tolist() # Menge der Planungsperioden (Wochen)

## Entscheidungsvariablen

In [ ]:
infinity = solver.infinity()

In [ ]:
# Produktionsmengen
Q={}
for p in P:
  for t in T: 
    Q[p,t] = solver.NumVar(0.0, infinity, f"{p},{t}")

In [ ]:
# Lagerbestand
B={}
for p in P:
  for t in T:
    B[p,t] = solver.NumVar(0.0, infinity, f"{p},{t}")

In [ ]:
# Rüstvariablen
R={}
for p in P: 
  for t in T:
        R[p,t] = solver.BoolVar(f"{p},{t}")

In [ ]:
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

## Parameter

In [ ]:
# Kosten
lc = kosten_zeiten.set_index(["Produkt"]).to_dict("dict")["Lagerkosten"]
rc = kosten_zeiten.set_index(["Produkt"]).to_dict("dict")["Rüstkosten"]

In [ ]:
# Zeiten
bz = kosten_zeiten.set_index(["Produkt"]).to_dict("dict")["Bearbeitungszeit"]
rz = kosten_zeiten.set_index(["Produkt"]).to_dict("dict")["Rüstzeit"]

In [ ]:
# Nachfrage
d = nachfrage.set_index(["Produkt","Woche"]).to_dict("dict")["Nachfrage"]

In [ ]:
# Kapazitäten 
cap = {}
for t in T:
  cap[t] = 5760

## Berechnung Lösung

In [ ]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Kosten) =', solver.Objective().Value())
else:
    print('Problem hat keine Lösung')

In [ ]:
for t in T:
  print(f"Woche: {t}")
  produzieren = sum(bz[p] * Q[p,t].solution_value() for p in P)
  rüsten = sum(rz[p] * R[p,t].solution_value() for p in P)
  print(f"Produktionszeit: {round(produzieren)}")
  print(f"Rüstzeit: {round(rüsten)}")
  print(f"Summe: {round(rüsten+produzieren)}")
  
  print("\n")
